In [1]:
import copy, math
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=2)

In [3]:
# X: Size (sqft), Number of Bedrooms, Number of floors, Age of Home
# y: Price (1000s dollars)

X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

In [19]:
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])

In [36]:
#@title Functions
predict = lambda x, w, b: np.dot(x, w) + b

def compute_cost(X, y, w, b):
    m = X.shape[0] # No. of training examples
    cost = 0.0
    for i  in range(m):
        f_wb_i = np.dot(X[i], w) + b
        cost = cost + (f_wb_i - y[i]) ** 2
    cost = cost / (2 * m)
    return cost

def compute_gradient(X, y, w, b):
    m, n = X.shape # No. of examples, features
    dJ_dw = np.zeros(n)
    dJ_db = 0.

    for i in range(m):
        err = predict(X[i], w, b) - y[i]
        for j in range(n):
            dJ_dw[j] = dJ_dw[j] + err * X[i, j]
        dJ_db += err
    dJ_dw /= m
    dJ_db /= m

    return dJ_dw, dJ_db

def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters):
    J_history = [] # For graphing later
    w = copy.deepcopy(w_in)
    b = b_in

    for i in range(num_iters):
        dJ_dw, dJ_db = compute_gradient(X, y, w, b)
        w = w - alpha * dJ_dw
        b = b - alpha * dJ_db
        if i < 100_000:
            J_history.append(cost_function(X, y, w, b))
        if i % math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}")
    return w, b, J_history

In [45]:
# Testing the implementation

initial_w = np.zeros_like(w_init)
initial_b = 0.

iterations = 1_000
alpha = 5.0e-7

w_final, b_final, J_history = gradient_descent(
    X = X_train,
    y = y_train,
    w_in = initial_w,
    b_in = initial_b,
    cost_function = compute_cost,
    gradient_function = compute_gradient,
    alpha = alpha,
    num_iters = iterations
)

Iteration    0: Cost  2529.46
Iteration  100: Cost   695.99
Iteration  200: Cost   694.92
Iteration  300: Cost   693.86
Iteration  400: Cost   692.81
Iteration  500: Cost   691.77
Iteration  600: Cost   690.73
Iteration  700: Cost   689.71
Iteration  800: Cost   688.70
Iteration  900: Cost   687.69
